# MEAN CENTER BEHAVIOR!
# Have to rerun everything when i have more subjects - as the group ICA needs to be re-estimated and then all that comes afterwards (which is essentially everything)

For future self:
* don't use the _Z files in .dr folders. Less recommended in my understanding. 


* to install packages and maybe run some stuff in the terminal I might need sometimes to run: conda activate myenv


# Import stuff + params

In [1]:
import os
import numpy as np

# need to set up to run fsl.
os.environ['FSLDIR'] = '/export/home/ranigera/fsl'
os.environ['PATH'] += ':/export/home/ranigera/fsl/bin'

group_ICA_path = '/export/home/ranigera/IDCH-rsfMRI/data/group_ICA'
rs_data_path = '/export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data'
rs_data_path_IDCH_sub_Nums = '/export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums'

input_files_file_name = 'input_files.txt'
input_files_txt_path = os.path.join(group_ICA_path, input_files_file_name)
n_ICs = 50 # number of independent components
TR = 0.70 # repetition time in seconds

# (original) file_format:
any_session_file_format = '_space-MNI152NLin6Asym_desc-smoothAROMAnonaggr_bold.nii.gz'

# R01 to IDCH mapping:
mapping_R01_to_IDCH = {'222': '101', '183': '102', '216': '103', '192': '104', '251': '105', '206': '106', '180': '107', '184': '108', '169': '109', '207': '110',
                       '159': '111', '115': '112', '114': '113', '232': '114', '173': '115', '171': '117', '215': '118', '265': '119', '177': '120', '269': '121',
                       '261': '122'}


# Preperation

## Copy data to be with IDCH sub numbers (and extract the brain)

In [2]:
# get all directories in the rs_data_path:
r01_sub_dirs = [f for f in os.listdir(rs_data_path) if os.path.isdir(os.path.join(rs_data_path, f))]
# iterate over all subjects:
for dir in r01_sub_dirs:
    sub_dir = os.path.join(rs_data_path, dir, 'func') 
    sub_R01_id = dir.split('Hab0')[1][:3] # get sub R01 id
    sub_IDCH_id = mapping_R01_to_IDCH[sub_R01_id] # get sub IDCH id

    # get subject's resting state:
    rs_files = [f for f in os.listdir(sub_dir) if any_session_file_format in f]
    for f in rs_files:
        # get the session number:
        session = f.split('rest_run-')[1][0]
        new_file_name = f'sub-{sub_IDCH_id}_ses-{session}_rest_MNI152-smoothAROMAnonaggr_bold_brain.nii.gz'
        # copy the file if not exists:
        if not os.path.exists(os.path.join(rs_data_path_IDCH_sub_Nums, new_file_name)):
            print(f'> Copying {f} to {new_file_name}')
            # os.system(f'cp {os.path.join(sub_dir, f)} {os.path.join(rs_data_path_IDCH_sub_Nums, new_file_name)}')
            os.system(f'fslmaths {os.path.join(sub_dir, f)} -mas $FSLDIR/data/standard/MNI152_T1_2mm_brain_mask.nii.gz {os.path.join(rs_data_path_IDCH_sub_Nums, new_file_name)}')
        # write the file name to the input_files.txt file:


> Copying sub-JODMBMFHab017301_task-rest_run-2_space-MNI152NLin6Asym_desc-smoothAROMAnonaggr_bold.nii.gz to sub-115_ses-2_rest_MNI152-smoothAROMAnonaggr_bold_brain.nii.gz
> Copying sub-JODMBMFHab022201_task-rest_run-2_space-MNI152NLin6Asym_desc-smoothAROMAnonaggr_bold.nii.gz to sub-101_ses-2_rest_MNI152-smoothAROMAnonaggr_bold_brain.nii.gz
> Copying sub-JODMBMFHab022201_task-rest_run-1_space-MNI152NLin6Asym_desc-smoothAROMAnonaggr_bold.nii.gz to sub-101_ses-1_rest_MNI152-smoothAROMAnonaggr_bold_brain.nii.gz
> Copying sub-JODMBMFHab018401_task-rest_run-1_space-MNI152NLin6Asym_desc-smoothAROMAnonaggr_bold.nii.gz to sub-108_ses-1_rest_MNI152-smoothAROMAnonaggr_bold_brain.nii.gz
> Copying sub-JODMBMFHab018401_task-rest_run-2_space-MNI152NLin6Asym_desc-smoothAROMAnonaggr_bold.nii.gz to sub-108_ses-2_rest_MNI152-smoothAROMAnonaggr_bold_brain.nii.gz
> Copying sub-JODMBMFHab021501_task-rest_run-1_space-MNI152NLin6Asym_desc-smoothAROMAnonaggr_bold.nii.gz to sub-118_ses-1_rest_MNI152-smoothAROMA

## Create input_files.txt file needed for the group ICA

In [3]:
# list all files in all subdirectories of rs_data_path_IDCH_sub_Nums
rs_data_files = []
for root, dirs, files in os.walk(rs_data_path_IDCH_sub_Nums):
    for file in files:
        if file.endswith('.nii.gz'):
            rs_data_files.append(os.path.join(root, file))
print('Number of files:', len(rs_data_files))
rs_data_files.sort()

with open(input_files_txt_path, 'w') as f:
    for file in rs_data_files:
        f.write(file + '\n')
rs_data_files

Number of files: 42


['/export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-101_ses-1_rest_MNI152-smoothAROMAnonaggr_bold_brain.nii.gz',
 '/export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-101_ses-2_rest_MNI152-smoothAROMAnonaggr_bold_brain.nii.gz',
 '/export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-102_ses-1_rest_MNI152-smoothAROMAnonaggr_bold_brain.nii.gz',
 '/export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-102_ses-2_rest_MNI152-smoothAROMAnonaggr_bold_brain.nii.gz',
 '/export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-103_ses-1_rest_MNI152-smoothAROMAnonaggr_bold_brain.nii.gz',
 '/export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-103_ses-2_rest_MNI152-smoothAROMAnonaggr_bold_brain.nii.gz',
 '/export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-104_ses-1_rest_MNI152-smoothAROMAnonaggr_bold_brain.nii.gz',
 '/export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/s

# Run group ICA (creates commad)

In [6]:
n_ICs=50 # number of independent components
print(f'melodic -i {input_files_txt_path} -o {group_ICA_path}/groupICA{n_ICs} --tr=0.70 --nobet -a concat --bgimage=$FSLDIR/data/standard/MNI152_T1_2mm_brain.nii.gz -m $FSLDIR/data/standard/MNI152_T1_2mm_brain_mask.nii.gz --report --Oall -d {n_ICs}')
n_ICs=40
print(f'melodic -i {input_files_txt_path} -o {group_ICA_path}/groupICA{n_ICs} --tr=0.70 --nobet -a concat --bgimage=$FSLDIR/data/standard/MNI152_T1_2mm_brain.nii.gz -m $FSLDIR/data/standard/MNI152_T1_2mm_brain_mask.nii.gz --report --Oall -d {n_ICs}')
n_ICs=30
print(f'melodic -i {input_files_txt_path} -o {group_ICA_path}/groupICA{n_ICs} --tr=0.70 --nobet -a concat --bgimage=$FSLDIR/data/standard/MNI152_T1_2mm_brain.nii.gz -m $FSLDIR/data/standard/MNI152_T1_2mm_brain_mask.nii.gz --report --Oall -d {n_ICs}')
n_ICs=20
print(f'melodic -i {input_files_txt_path} -o {group_ICA_path}/groupICA{n_ICs} --tr=0.70 --nobet -a concat --bgimage=$FSLDIR/data/standard/MNI152_T1_2mm_brain.nii.gz -m $FSLDIR/data/standard/MNI152_T1_2mm_brain_mask.nii.gz --report --Oall -d {n_ICs}')
# Determine # of components automatically (tend to create a LOT of components):
# print(f'melodic -i {input_files_txt_path} -o {group_ICA_path}/groupICA{"auto_n_ICs"} --tr=0.70 --nobet -a concat --bgimage=$FSLDIR/data/standard/MNI152_T1_2mm_brain.nii.gz -m $FSLDIR/data/standard/MNI152_T1_2mm_brain_mask.nii.gz --report --Oall')


melodic -i /export/home/ranigera/IDCH-rsfMRI/data/group_ICA/input_files.txt -o /export/home/ranigera/IDCH-rsfMRI/data/group_ICA/groupICA50 --tr=0.70 --nobet -a concat --bgimage=$FSLDIR/data/standard/MNI152_T1_2mm_brain.nii.gz -m $FSLDIR/data/standard/MNI152_T1_2mm_brain_mask.nii.gz --report --Oall -d 50
melodic -i /export/home/ranigera/IDCH-rsfMRI/data/group_ICA/input_files.txt -o /export/home/ranigera/IDCH-rsfMRI/data/group_ICA/groupICA40 --tr=0.70 --nobet -a concat --bgimage=$FSLDIR/data/standard/MNI152_T1_2mm_brain.nii.gz -m $FSLDIR/data/standard/MNI152_T1_2mm_brain_mask.nii.gz --report --Oall -d 40
melodic -i /export/home/ranigera/IDCH-rsfMRI/data/group_ICA/input_files.txt -o /export/home/ranigera/IDCH-rsfMRI/data/group_ICA/groupICA30 --tr=0.70 --nobet -a concat --bgimage=$FSLDIR/data/standard/MNI152_T1_2mm_brain.nii.gz -m $FSLDIR/data/standard/MNI152_T1_2mm_brain_mask.nii.gz --report --Oall -d 30
melodic -i /export/home/ranigera/IDCH-rsfMRI/data/group_ICA/input_files.txt -o /expor

# Run dual regression (creates command)

* This is dual regression without the 3rd step of group analysis. This is done separately later)

In [7]:
n_ICs=20 # number of independent components
print(f'dual_regression {group_ICA_path}/groupICA{n_ICs}/melodic_IC 1 \
      -1 0 \
      {group_ICA_path}/groupICA{n_ICs}.dr {" ".join(item for item in rs_data_files)}')

dual_regression /export/home/ranigera/IDCH-rsfMRI/data/group_ICA/groupICA20/melodic_IC 1       -1 0       /export/home/ranigera/IDCH-rsfMRI/data/group_ICA/groupICA20.dr /export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-101_ses-1_rest_MNI152-smoothAROMAnonaggr_bold_brain.nii.gz /export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-101_ses-2_rest_MNI152-smoothAROMAnonaggr_bold_brain.nii.gz /export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-102_ses-1_rest_MNI152-smoothAROMAnonaggr_bold_brain.nii.gz /export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-102_ses-2_rest_MNI152-smoothAROMAnonaggr_bold_brain.nii.gz /export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-103_ses-1_rest_MNI152-smoothAROMAnonaggr_bold_brain.nii.gz /export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-103_ses-2_rest_MNI152-smoothAROMAnonaggr_bold_brain.nii.gz /export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH

## Averaging each subect's data for each IC

In [30]:
n_ICs=30

stage2_ic_file_prefix = 'dr_stage2_ic' # move to params
# if not exists, create the directory:

sub_avg_folder = os.path.join(group_ICA_path, f'groupICA{n_ICs}.sub_avg')

if not os.path.exists(sub_avg_folder):
    os.makedirs(sub_avg_folder)

for IC in range(n_ICs):
    print(f'Processing IC {IC}')
    # get the IC into a string that has 4 digits:
    IC_str = str(IC).zfill(4)
    file_name = f'{stage2_ic_file_prefix}{IC_str}.nii.gz'
    file_path = os.path.join(group_ICA_path, f'groupICA{n_ICs}.dr', file_name)
    # split the IC into sub-ICs:
    # check first if exists (anything with {sub_avg_folder}/IC{IC_str}_):
    if not os.path.exists(f'{sub_avg_folder}/IC{IC_str}_0000.nii.gz'):
        os.system(f'fslsplit {file_path} {sub_avg_folder}/IC{IC_str}_ -t')
    else:
        print(f'IC{IC_str} already split into sub-ICs')
    # get all the files in the sub_avg_folder and iterate them in pairs:
    sub_IC_files = [f for f in os.listdir(sub_avg_folder) if f.startswith(f'IC{IC_str}_')]
    sub_IC_files.sort()
    # iterate over the sub_IC_files in pairs and concatenate them:
    for i in range(0, len(sub_IC_files), 2):
        sub_file1 = os.path.join(sub_avg_folder, sub_IC_files[i])
        sub_file2 = os.path.join(sub_avg_folder, sub_IC_files[i+1])
        # check if the concatenated file exists:
        if not os.path.exists(f'{sub_avg_folder}/IC{IC_str}_avg_{i//2}.nii.gz'):
            os.system(f'fslmaths {sub_file1} -add {sub_file2} -div 2 {sub_avg_folder}/IC{IC_str}_avg_{i//2}.nii.gz') # this calculate the average in the right way. I TESTED IT!
        else:
            print(f'IC{IC_str}_avg_{i//2}.nii.gz already exists')


        break
    break



# group_ICA_path}/groupICA{n_ICs}.dr

Processing IC 0
Processing IC 1
Processing IC 2
Processing IC 3
Processing IC 4
Processing IC 5
Processing IC 6
Processing IC 7
Processing IC 8
Processing IC 9
Processing IC 10
Processing IC 11
Processing IC 12
Processing IC 13
Processing IC 14
Processing IC 15
Processing IC 16
Processing IC 17
Processing IC 18
Processing IC 19
Processing IC 20
Processing IC 21
Processing IC 22
Processing IC 23
Processing IC 24
Processing IC 25
Processing IC 26
Processing IC 27
Processing IC 28
Processing IC 29


# Archive

## Create design matrix and con file (random for now, replace with actual behavior)

In [ ]:
# TEMP TEMP TEMP
num_files = len(rs_data_files)
# Generate random numbers for the EV
ev_data = np.random.rand(num_files)

# Prepare the .mat file content
header = f"""/NumWaves\t1
/NumPoints\t{num_files}

/Matrix
"""

# Format the matrix content
matrix_content = "\n".join([f"{value}\t" for value in ev_data])

# Combine header and matrix content
design_matrix_content = header + matrix_content
# Write the design matrix to a file
output_file = "behav_corr_design_matrix.mat"
output_file = os.path.join(group_ICA_path, output_file)
with open(output_file, 'w') as file:
    file.write(design_matrix_content)

print(f"Design matrix file '{output_file}' created successfully!")


Design matrix file '/export/home/ranigera/IDCH-rsfMRI/data/group_ICA/behav_corr_design_matrix.mat' created successfully!


In [ ]:
# TEMP TEMP TEMP
# Parameters for the contrast file
contrast_name1 = "Pos corr"
contrast_name2 = "Neg corr"
num_waves = 1  # As per your new design, only one EV
num_contrasts = 2  # Two contrasts: positive and negative

# Matrix for contrasts
# Positive correlation: 1 for the EV
# Negative correlation: -1 for the EV
contrast_matrix = [
    [1.0],  # Positive correlation
    [-1.0]  # Negative correlation
]

# Create the .con file content
header = f"""/ContrastName1\t{contrast_name1}
/ContrastName2\t{contrast_name2}
/NumWaves\t{num_waves}
/NumContrasts\t{num_contrasts}

/Matrix
"""

# Format the matrix content
matrix_content = "\n".join(["\t".join([f"{value}" for value in row]) for row in contrast_matrix])

# Combine header and matrix content
contrast_matrix_content = header + matrix_content

# Write the contrast matrix to a file
output_file = "behav_corr_design_matrix.con"
output_file = os.path.join(group_ICA_path, output_file)
with open(output_file, 'w') as file:
    file.write(contrast_matrix_content)
print(contrast_matrix_content)
print(f"Contrast matrix file '{output_file}' created successfully!")


/ContrastName1	Pos corr
/ContrastName2	Neg corr
/NumWaves	1
/NumContrasts	2

/Matrix
1.0
-1.0
Contrast matrix file '/export/home/ranigera/IDCH-rsfMRI/data/group_ICA/behav_corr_design_matrix.con' created successfully!


In [ ]:
n_ICs=50 # number of independent components
print(f'dual_regression {group_ICA_path}/groupICA{n_ICs}/melodic_IC 1 \
      {group_ICA_path}/behav_corr_design_matrix.mat {group_ICA_path}/behav_corr_design_matrix.con 5000 \
      {group_ICA_path}/groupICA{n_ICs}.dr {" ".join(item for item in rs_data_files)}')


dual_regression /export/home/ranigera/IDCH-rsfMRI/data/group_ICA/groupICA50/melodic_IC 1       /export/home/ranigera/IDCH-rsfMRI/data/group_ICA/behav_corr_design_matrix.mat /export/home/ranigera/IDCH-rsfMRI/data/group_ICA/behav_corr_design_matrix.con 5000       /export/home/ranigera/IDCH-rsfMRI/data/group_ICA/groupICA50.dr /export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-101_ses-1_rest_MNI152-smoothAROMAnonaggr_bold.nii.gz /export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-101_ses-2_rest_MNI152-smoothAROMAnonaggr_bold.nii.gz /export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-102_ses-1_rest_MNI152-smoothAROMAnonaggr_bold.nii.gz /export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-102_ses-2_rest_MNI152-smoothAROMAnonaggr_bold.nii.gz /export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-103_ses-1_rest_MNI152-smoothAROMAnonaggr_bold.nii.gz /export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_su

In [ ]:
n_ICs=30 # number of independent components
print(f'dual_regression {group_ICA_path}/groupICA{n_ICs}/melodic_IC 1 \
      -1 0 \
      {group_ICA_path}/groupICA{n_ICs}.dr {" ".join(item for item in rs_data_files)}')


dual_regression /export/home/ranigera/IDCH-rsfMRI/data/group_ICA/groupICA30/melodic_IC 1       -1 0       /export/home/ranigera/IDCH-rsfMRI/data/group_ICA/groupICA30.dr /export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-101_ses-1_rest_MNI152-smoothAROMAnonaggr_bold.nii.gz /export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-101_ses-2_rest_MNI152-smoothAROMAnonaggr_bold.nii.gz /export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-102_ses-1_rest_MNI152-smoothAROMAnonaggr_bold.nii.gz /export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-102_ses-2_rest_MNI152-smoothAROMAnonaggr_bold.nii.gz /export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-103_ses-1_rest_MNI152-smoothAROMAnonaggr_bold.nii.gz /export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-103_ses-2_rest_MNI152-smoothAROMAnonaggr_bold.nii.gz /export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-104_ses-1_rest_MNI152-s

In [ ]:
n_ICs=40 # number of independent components
print(f'dual_regression {group_ICA_path}/groupICA{n_ICs}/melodic_IC 1 \
      {group_ICA_path}/behav_corr_design_matrix.mat {group_ICA_path}/behav_corr_design_matrix.con 1 \
      {group_ICA_path}/groupICA{n_ICs}.dr {" ".join(item for item in rs_data_files)}')


dual_regression /export/home/ranigera/IDCH-rsfMRI/data/group_ICA/groupICA40/melodic_IC 1       /export/home/ranigera/IDCH-rsfMRI/data/group_ICA/behav_corr_design_matrix.mat /export/home/ranigera/IDCH-rsfMRI/data/group_ICA/behav_corr_design_matrix.con 1       /export/home/ranigera/IDCH-rsfMRI/data/group_ICA/groupICA40.dr /export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-101_ses-1_rest_MNI152-smoothAROMAnonaggr_bold.nii.gz /export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-101_ses-2_rest_MNI152-smoothAROMAnonaggr_bold.nii.gz /export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-102_ses-1_rest_MNI152-smoothAROMAnonaggr_bold.nii.gz /export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-102_ses-2_rest_MNI152-smoothAROMAnonaggr_bold.nii.gz /export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-103_ses-1_rest_MNI152-smoothAROMAnonaggr_bold.nii.gz /export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNu

In [ ]:
n_ICs=40 # number of independent components
print(f'dual_regression {group_ICA_path}/groupICA{n_ICs}/melodic_IC 1 \
      {group_ICA_path}/behav_corr_design_matrix.mat {group_ICA_path}/behav_corr_design_matrix.con 0 \
      {group_ICA_path}/groupICA{n_ICs}.dr {" ".join(item for item in rs_data_files)}')


dual_regression /export/home/ranigera/IDCH-rsfMRI/data/group_ICA/groupICA40/melodic_IC 1       /export/home/ranigera/IDCH-rsfMRI/data/group_ICA/behav_corr_design_matrix.mat /export/home/ranigera/IDCH-rsfMRI/data/group_ICA/behav_corr_design_matrix.con 0       /export/home/ranigera/IDCH-rsfMRI/data/group_ICA/groupICA40.dr /export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-101_ses-1_rest_MNI152-smoothAROMAnonaggr_bold.nii.gz /export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-101_ses-2_rest_MNI152-smoothAROMAnonaggr_bold.nii.gz /export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-102_ses-1_rest_MNI152-smoothAROMAnonaggr_bold.nii.gz /export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-102_ses-2_rest_MNI152-smoothAROMAnonaggr_bold.nii.gz /export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-103_ses-1_rest_MNI152-smoothAROMAnonaggr_bold.nii.gz /export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNu